# Error Measurement

So far, we have used a variety of measures to compare models or judge how well a model performed its task. Now, we will analyze best practices for judging the accuracy of forecasts, emphasizing the specific issues regarding **time series** data.

For those new to time series forecasting, it is most important to understand that standard cross-validation is typically not recommended. It is not possible to select randomly sampled training, validation, and testing data sets for each of these categories in a time-independent manner.

However, things are even more complicated. You need to think about how different data samples relate to each other over time, even though they appear independent. For example, suppose you are working on a **time series** classification task, so that you have many separate **time series** samples, each of which is its own data point. It may be tempting to think that in this case it is possible to randomly choose **time series** for each training, validation and test set, but this does not work. The problem with this approach is that it does not reflect how you would use your model, i.e. it would not reflect training your model on earlier data nor testing it on later data.

